Image Recognition
===

This notebook will create a convolutional neural network to classify images in either the mnist or cifar-10 datasets.

In [1]:
# Tensorflow and numpy to create the neural network
import tensorflow as tf
import numpy as np

# Matplotlib to plot info to show our results
import matplotlib.pyplot as plt


# OS to load files and save checkpoints|
import os

#%matplotlib inline

Loading the data
---

This code will load the dataset that you'll use to train and test the model.

The code provided will load the mnist or cifar data from files, you'll need to add the code that processes it into a format your neural network can use.

MNIST
---

Run this cell to load mnist data.

In [2]:
# Load MNIST data from tf examples



    

CIFAR-10
---

Run this cell to load cifar-10 data

In [3]:
# Load cifar data from file

image_height = 32
image_width = 32

color_channels = 3

model_name = "cifar"

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar_path = './cifar-100-python/'

train_data = np.array([])
train_labels = np.array([])

# Load all the data batches.
data_batch = unpickle(cifar_path + 'train')
print(data_batch.keys())


train_data = data_batch[b'data']
train_labels = data_batch[b'coarse_labels']


# Load the eval batch.
eval_batch = unpickle(cifar_path + 'test')

eval_data = eval_batch[b'data']
eval_labels = eval_batch[b'coarse_labels'] 

# Load the english category names.
category_names_bytes = unpickle(cifar_path + 'meta')[b'coarse_label_names']
category_names = list(map(lambda x: x.decode("utf-8"), category_names_bytes))

# TODO: Process Cifar data
def process_data(data):
    float_data = np.array(data, dtype = float) / 255.0
    
    reshaped_data = np.reshape(float_data, (-1, color_channels, image_height, image_width))
    
    transposed_data = np.transpose(reshaped_data, [0, 2, 3, 1])
    #plt.imshow(transposed_data[0])
    
    return transposed_data
            
train_data = process_data(train_data)
eval_data = process_data(eval_data)


print("training data length:",len(train_data))
print("eval_data:",len(eval_data))

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
training data length: 50000
eval_data: 10000


Once the data is processed, you have a few variables for the data itself and info about its shape:

### Model Info

- **image_height, image_width** - The height and width of the processed images
- **color_channels** - the number of color channels in the image. This will be either 1 for grayscale or 3 for rgb.
- **model_name** - either "cifar" or "mnist" - if you need to handle anything differently based on the model, check this variable.
- **category_names** - strings for each category name (used to print out labels when testing results)

### Training Data

- **train_data** - the training data images
- **train_labels** - the labels for the training data - the "answer key"

### Evaluation Data

- **eval_data** - Image data for evaluation. A different set of images to test your network's effectiveness.
- **eval_labels** - the answer key for evaluation data.

Building the Neural Network Model
--

Next, you'll build a neural network with the following architecture:

- An input placeholder that takes one or more images.
- 1st Convolutional layer with 32 filters and a kernel size of 5x5 and same padding
- 1st Pooling layer with a 2x2 pool size and stride of 2
- 2nd Convolutional layer with 64 filters and a kernel size of 5x5 and same padding
- 2nd Pooling layer with a 2x2 pool size and stride of 2
- Flatten the pooling layer
- A fully connected layer with 1024 units
- A dropout layer with a rate of 0.4
- An output layer with an output size equal to the number of labels.

In [4]:
# TODO: The neural network
class ConvNet:
    def __init__(self, image_height, image_width, channels, num_classes):
        self.input_layer = tf.placeholder(dtype=tf.float32, shape = [None, image_height, image_width, channels], name='inputs')
        print(self.input_layer.shape)
        
        self.labels = tf.placeholder(dtype=tf.float32, shape = [None, num_classes])
        
        conv_layer_1 = tf.layers.conv2d(self.input_layer, filters = 32, kernel_size=[5, 5], padding='same', activation = tf.nn.relu)
        print(conv_layer_1)
        
        pooling_layer_1 = tf.layers.max_pooling2d(conv_layer_1, pool_size=[2, 2], strides = 2)
        print(pooling_layer_1.shape)
        
        conv_layer_2 = tf.layers.conv2d(pooling_layer_1, filters = 64, kernel_size=[5, 5], padding='same', activation = tf.nn.relu)
        print(conv_layer_2)
        
        pooling_layer_2 = tf.layers.max_pooling2d(conv_layer_2, pool_size=[2, 2], strides = 2)
        print(pooling_layer_2.shape)
        
        flattened_pooling = tf.layers.flatten(pooling_layer_2)
        dense_layer = tf.layers.dense(flattened_pooling, 1024, activation=tf.nn.relu)
        print(dense_layer.shape)
        
        dropout = tf.layers.dropout(dense_layer, rate = 0.4, training=True)
        
        outputs = tf.layers.dense(dropout, num_classes)
        print(outputs.shape)
        
        self.choice = tf.argmax(outputs, axis=1)
        self.probability = tf.nn.softmax(outputs)
        
        self.labels = tf.placeholder(dtype=tf.float32, name = 'labels')
        self.accuracy, self.accuracy_op = tf.metrics.accuracy(self.labels, self.choice)
        
        one_hot_labels = tf.one_hot(indices=tf.cast(self.labels, dtype=tf.int32), depth=num_classes)
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_labels, logits=outputs)
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-2)
        self.train_operation = optimizer.minimize(loss=self.loss, global_step=tf.train.get_global_step())
        
CNN = ConvNet(32,32,3,20)

(?, 32, 32, 3)
Tensor("conv2d/Relu:0", shape=(?, 32, 32, 32), dtype=float32)
(?, 16, 16, 32)
Tensor("conv2d_1/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
(?, 8, 8, 64)
(?, 1024)
(?, 20)


The Training Process
---

The cells below will set up and run the training process.

- Set up initial values for batch size, training length.
- Process data into batched datasets to feed into the network.
- Run through batches of training data, update weights, save checkpoints.

In [5]:
# TODO: initialize variables
training_steps = 20000
batch_size = 100

path = '/' + model_name + '-cnn/'

load_checkpoint = False

performance_graph = np.array([])


In [ ]:
# TODO: implement the training loop
tf.reset_default_graph()
dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
dataset = dataset.shuffle(buffer_size = len(train_labels))
dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_iterator = dataset.make_initializable_iterator()

next_element = dataset_iterator.get_next()

cnn = ConvNet (image_height, image_width, color_channels, 20)

saver = tf.train.Saver(max_to_keep=2)

if not os.path.exists(path):
    os.makedirs(path)
    
with tf.Session() as sess:
    
    if load_checkpoint:
        checkpoint = tf.train.get_checkpoint_state(path)
        saver.restore(sess, checkpoint.model_checkpoint_path)
        
    else:
        sess.run(tf.global_variables_initializer())
        
    sess.run(tf.local_variables_initializer())
    sess.run(dataset_iterator.initializer)

    for step in range(training_steps):
        current_batch = sess.run(next_element)
        
        batch_inputs = current_batch[0]
        batch_labels = current_batch[1]
        
        sess.run((cnn.train_operation, cnn.accuracy_op), feed_dict = {cnn.input_layer: batch_inputs, cnn.labels: batch_labels})       
        
        if step % 10 == 0:
            performance_graph = np.append(performance_graph, sess.run(cnn.accuracy))
            
        if step % 100 == 0 and step > 0:
            current_acc = sess.run(cnn.accuracy)
        
            print("Accuracy at step " + str(step) + ": " + str(current_acc))
            print("Saving checkpoint")
            saver.save(sess, path + model_name, step)
                 
    print("Saving final checkpoint for training session.")
    saver.save(sess, path + model_name, step)

(?, 32, 32, 3)
Tensor("conv2d/Relu:0", shape=(?, 32, 32, 32), dtype=float32)
(?, 16, 16, 32)
Tensor("conv2d_1/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
(?, 8, 8, 64)
(?, 1024)
(?, 20)
Accuracy at step 100: 0.0654455
Saving checkpoint
Accuracy at step 200: 0.080398
Saving checkpoint
Accuracy at step 300: 0.0936877
Saving checkpoint
Accuracy at step 400: 0.105611
Saving checkpoint
Accuracy at step 500: 0.116407
Saving checkpoint
Accuracy at step 600: 0.126273
Saving checkpoint
Accuracy at step 700: 0.134565
Saving checkpoint
Accuracy at step 800: 0.142222
Saving checkpoint
Accuracy at step 900: 0.149367
Saving checkpoint
Accuracy at step 1000: 0.155564
Saving checkpoint
Accuracy at step 1100: 0.161662
Saving checkpoint
Accuracy at step 1200: 0.167377
Saving checkpoint
Accuracy at step 1300: 0.172744
Saving checkpoint
Accuracy at step 1400: 0.177345
Saving checkpoint
Accuracy at step 1500: 0.182278
Saving checkpoint
Accuracy at step 1600: 0.18722
Saving checkpoint
Accuracy at step 17

Accuracy at step 16100: 0.414267
Saving checkpoint
Accuracy at step 16200: 0.415242
Saving checkpoint
Accuracy at step 16300: 0.416178
Saving checkpoint
Accuracy at step 16400: 0.417099
Saving checkpoint
Accuracy at step 16500: 0.417998
Saving checkpoint
Accuracy at step 16600: 0.418924
Saving checkpoint
Accuracy at step 16700: 0.419885
Saving checkpoint
Accuracy at step 16800: 0.420801
Saving checkpoint
Accuracy at step 16900: 0.421721
Saving checkpoint
Accuracy at step 17000: 0.422589
Saving checkpoint
Accuracy at step 17100: 0.423574
Saving checkpoint
Accuracy at step 17200: 0.424485
Saving checkpoint
Accuracy at step 17300: 0.4254
Saving checkpoint
Accuracy at step 17400: 0.426308
Saving checkpoint
Accuracy at step 17500: 0.427191
Saving checkpoint
Accuracy at step 17600: 0.428177
Saving checkpoint
Accuracy at step 17700: 0.429179
Saving checkpoint
Accuracy at step 17800: 0.430047
Saving checkpoint
Accuracy at step 17900: 0.430987
Saving checkpoint
Accuracy at step 18000: 0.431904


Evaluating Performance
---

These cells will evaluate the performance of your network!

In [ ]:
# TODO: Display graph of performance over time
plt.plot(performance_graph)
plt.figure().set_facecolor('white')
plt.xlabel("Steps/10")
plt.ylabel("Accuracy")


In [ ]:
# TODO: Run through the evaluation data set, check accuracy of model
with tf.Session() as sess:
    checkpoint = tf.train.get_checkpoint_state(path)
    saver.restore(sess, checkpoint.model_checkpoint_path)
    
    sess.run(tf.local_variables_initializer())
    
    for image, label in zip(eval_data, eval_labels):
        sess.run(cnn.accuracy_op, feed_dict= {cnn.input_layer: [image], cnn.labels:label})
    
    print(sess.run(cnn.accuracy))

In [ ]:
# TODO: Get a random set of images and make guesses for each
with tf.Session() as sess:
    checkpoint = tf.train.get_checkpoint_state(path)
    saver.restore(sess,checkpoint.model_checkpoint_path)

    indexes = np.random.choice(len(eval_data), 10, replace = False)

    rows = 5
    cols = 2
     
    fig, axes = plt.subplots(rows, cols, figsize=(5,5))
    fig.patch.set_facecolor('white')
    image_count = 0
     
    for idx in indexes:
        image_count += 1
        sub = plt.subplot(rows,cols,image_count)
        img = eval_data[idx]
        if model_name == "mnist":
            img = img.reshape(28, 28)
        plt.imshow(img)
        guess = sess.run(cnn.choice, feed_dict={cnn.input_layer:[eval_data[idx]]})
        if model_name == "mnist":
            guess_name = str(guess[0])
            actual_name = str(eval_labels[idx])
        else:
            guess_name = category_names[guess[0]]
            actual_name = category_names[eval_labels[idx]]
        sub.set_title("G: " + guess_name + " A: " + actual_name)
    plt.tight_layout()